## Test of the targeted controller for std1

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import os
import sys
from loguru import logger

In [10]:
vimms_folder = '/Users/simon/git/vimms'
sys.path.append(vimms_folder)

In [127]:
from vimms.Chemicals import ChemicalMixtureFromMZML
from vimms.Controller import create_targets_from_toxid, TargetedController
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Environment import Environment
from vimms.Common import POSITIVE, set_log_level_warning
from vimms.Roi import RoiParams

## Set the path to the experimental folder to get the mzML

In [12]:
experimental_results_folder = '/Users/simon/clms_er'
data_folder = os.path.join(experimental_results_folder,'20200904_DIA_test','02_DIA_experiments_stdmix')

In [14]:
standard = 'std1'
mzml_file = os.path.join(data_folder,'{}_Fullscan.mzML'.format(standard))

## Setup RIO parameters for the mzml -> chemicals process and create chemicals

In [103]:
rp = RoiParams(min_intensity=500)
cm = ChemicalMixtureFromMZML(mzml_file, roi_params=rp)
dataset = cm.sample(None, 2)

## Load the toxid file and make the target list

In [128]:
toxid_file = os.path.join(vimms_folder,'tests','fixtures','StdMix1_pHILIC_Current.csv')
target_list = create_targets_from_toxid(toxid_file, rt_delta=120., mz_delta=20)
print(len(target_list))

219


## Setup the controller, mass spec and environment

In [129]:
ce_values = [10, 20, 30]
n_replicates = 2
ionisation_mode = POSITIVE
controller = TargetedController(target_list, ce_values, n_replicates=n_replicates, limit_acquisition=False, min_ms1_intensity=500)
ms = IndependentMassSpectrometer(ionisation_mode, dataset, None)
env = Environment(ms, controller, 0, 1600)

## Run the simulation

In [130]:
set_log_level_warning()
env.run()

(1600.000s) ms_level=1: 100%|█████████▉| 1599.6000000001948/1600 [00:59<00:00, 26.96it/s]


## Summarise the targets -- what has been hit

In [132]:
controller.summarise_activity(print)

Summary of targets
Melatonin[M+H]+ (m/z: 233.12379164668644->233.13311678485505, rt: 137.39999999999998->377.4)
	10 -> 35 scans
	20 -> 34 scans
	30 -> 34 scans
Melatonin[M+K]+ (m/z: 271.07891417705343->271.089757550488, rt: 137.39999999999998->377.4)
	10 -> 12 scans
	20 -> 12 scans
	30 -> 12 scans
Melatonin[M+Na]+ (m/z: 255.10529625580142->255.11550067174, rt: 137.39999999999998->377.4)
	10 -> 15 scans
	20 -> 14 scans
	30 -> 14 scans
Phenylhydrazine[M+H]+ (m/z: 109.07384319829636->109.07820623928511, rt: 168.0->408.0)
	10 -> 11 scans
	20 -> 11 scans
	30 -> 11 scans
Phenylhydrazine[M+K]+ (m/z: 147.02896572866337->147.03484700491805, rt: 168.0->408.0)
	10 -> 4 scans
	20 -> 4 scans
	30 -> 4 scans
Phenylhydrazine[M+Na]+ (m/z: 131.05534780741135->131.06059012617004, rt: 168.0->408.0)
	10 -> 12 scans
	20 -> 12 scans
	30 -> 12 scans
Nicotinate[M+H]+ (m/z: 124.03682407071359->124.04178564290787, rt: 317.4->557.4)
	10 -> 31 scans
	20 -> 30 scans
	30 -> 30 scans
Nicotinate[M+K]+ (m/z: 161.991946

In [133]:
env.write_mzML(None, 'test.mzML')